In [24]:
from langchain.llms import OpenAI
import os
from langchain import huggingfacehub

In [25]:
llm= OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],temperature=0.6)

In [26]:
try:
    response = llm.predict("What is the capital of the USA?")
    print(response)
except Exception as e:
    print(f"Error: {e}")

Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]="hf_zjftfTRKFEebywxfIoyKUwepABtfGJS"

llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})
output=llm_huggingface.predict("Can you tell me the capital of Russia")
print(output)

## Prompt Templates And LLMChain

In [ ]:
from langchain.prompts import PromptTemplate

#input variables and template
prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="France")

In [ ]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("India"))

## Combining Multiple Chains Uing simple Sequential Chain

In [ ]:
#first chain for capital template
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)


# second chain for famous template
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template)

In [ ]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("Spain")

## Sequential Chain

In [ ]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")
capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")


famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")
famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

chain({'country':"England"})

# Chatmodels With ChatOpenAI

In [ ]:
from langchain.chat_models import ChatOpenAI

#3 main  messages: 
#HumanMessage: Represents user input (e.g., "What's the capital of France?").
#SystemMessage: Provides system instructions for the AI (e.g., "You are a helpful assistant.").
#AIMessage: Represents the AI’s response (e.g., "The capital of France is Paris.").

from langchain.schema import HumanMessage,SystemMessage,AIMessage

chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')


chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])


## Prompt Template + LLM +Output Parsers

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [ ]:
system_template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",system_template),
    ("human",human_template)
])

chain=chatprompt|chatllm|Commaseperatedoutput()
chain.invoke({"text":"intelligent"})